In [3]:
import pandas as pd
import numpy as np
import datetime as dt
import time
import matplotlib.pyplot as plt
from matplotlib import animation
from sim_functions_ver6 import *

allShiftsDF = pd.read_csv("ver6/4carTest.csv", sep=";", index_col=None)
filename = '4cars_test'

def handleDF(sim):
    df = sim.copy(deep=True)
    df = df.set_index(['time','car'])
    return df.T.stack().T
    
def plotFigures(name, sim, algo):
    testDF = handleDF(sim)
    
    fig4, ax4 = plt.subplots(figsize=(80,16))
    
    for i in range(8):
        ax4.scatter(testDF.index, np.ones(len(testDF.index))*i, 
                    s=(testDF['batt'][i]**2.1).apply(int))

    for j in range(len(testDF.index)):
        if j % 24 == 0:
            ax4.axvline(x=testDF.index[j], ymin=-1, ymax=1, 
                    color='black', linestyle='-', lw=0.7, alpha=0.5)
        else:
            ax4.axvline(x=testDF.index[j], ymin=-1, ymax=1, 
                    color='black', linestyle='-', lw=0.5, alpha=0.2)

    ax4.yaxis.set_ticks(np.arange(0,8,1))
    ax4.set_title('Batt (kW)', {'fontsize': 20})
    ax4.set_xlim([dt.datetime(2019,1,1,5,0,0), dt.datetime(2019,1,6,6,0,0)])
    plt.savefig('test/' + name + '_' + algo + '_rate.png')

###################################################################################################################
chunks = getChunks()
startTime = readTime("2019-01-01 06:00:00")
runTime = 24*5                  # CHOOSE RUNTIME (HRS)
mpkw = 4                        # SET AVERAGE MILES PER kW THAT WILL DETERMINE RATE OF BATT DECREASE
mph = 16                        # SET AVERAGE MILES PER HR COVERED
car_cols = ["battPerc","inDepot","battSize","chargePt"]
cp_cols = ["maxRate","inUse"]
sim_cols = ['time','car','charge_rate','batt','event']


######################## PARAMETERS (4 CARS) ############################
chargeCapacity = 12             # SET MAX AVAILABLE POWER IN CENTRE (kW/hr)
carData = [[30, 1, 30, 0], [30, 1, 30, 1], [30, 1, 30, 2], [30, 1, 30, 3]]
chargePtData = [[7, 1], [7, 1], [7, 1], [7, 1]]

############### RUN SIMULATION OVER EVERY DAY IN CSV ####################

dumbDF, dumb_sim, dumb_rc = runSimulation(
                        startTime, runTime,
                        carData, car_cols, allShiftsDF,
                        chargePtData, cp_cols, chargeCapacity,
                        sim_cols, mph, mpkw, 'dumbCharge')

smart_leavetimeDF, smart_leavetime_sim, smart_leavetime_rc = runSimulation(
                        startTime, runTime,
                        carData, car_cols, allShiftsDF,
                        chargePtData, cp_cols, chargeCapacity,
                        sim_cols, mph, mpkw, 'smartCharge_leavetime')

smart_battDF, smart_batt_sim, smart_batt_rc = runSimulation(
                        startTime, runTime,
                        carData, car_cols, allShiftsDF,
                        chargePtData, cp_cols, chargeCapacity,
                        sim_cols, mph, mpkw, 'smartCharge_batt')

smartDF, smart_sim, smart_rc = runSimulation(
                        startTime, runTime,
                        carData, car_cols, allShiftsDF,
                        chargePtData, cp_cols, chargeCapacity,
                        sim_cols, mph, mpkw, 'superSmartCharge')

###############################################################
# SAVE TO PNG (ONLY RUN WHEN ALL ALGORITHMS ARE UNCOMMENTED)
# NOTE: CREATE A FOLDER CALLED 'FIGURES' FIRST
###############################################################
# plotFigures(filename, dumb_sim, 'dumb')
# plotFigures(filename, smart_leavetime_sim, 'leavetime')
# plotFigures(filename, smart_batt_sim, 'batt')
# plotFigures(filename, smart_sim, 'smart')
# plt.close('all')

In [43]:
# fleettype = "4_small"
fleettype = "4_medium"
# fleettype = "4_large"
# fleettype = "4_small_reduced_cps"
# fleettype = "4_medium_reduced_cps"
# fleettype = "4_large_reduced_cps"
# fleettype = "8_small"
# fleettype = "8_medium"
# fleettype = "8_large"
# fleettype = "8_small_reduced_cps"
# fleettype = "8_medium_reduced_cps"
# fleettype = "8_large_reduced_cps"

def getData(df, search, column):
    return df.loc[df.type == search, column].values.tolist()[0]

fleetdata = pd.read_csv("ver6/fleetData.csv", sep=";", index_col=None)

chargeCapacity = getData(fleetdata, fleettype, 'chargeCapacity')
carData = eval(getData(fleetdata, fleettype, 'carData'))
chargePtData = eval(getData(fleetdata, fleettype, 'chargePtData'))